In [70]:
import numpy as np
import pandas as pd
from scipy.linalg import expm
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui
%gui qt5
import matplotlib.pyplot as plt

from scipy.spatial import distance
import time

### Plotting

In [54]:
#set up plottig GUI
app = QtGui.QApplication([])
pg.setConfigOption('background','w')  

In [56]:
win = pg.GraphicsWindow(title="Occupancy Detection GUI")
plot1 = win.addPlot()
plot1.setXRange(-6,6)
plot1.setYRange(0,6)
plot1.setLabel('left',text = 'Y position (m)')
plot1.setLabel('bottom', text= 'X position (m)')
s1 = plot1.plot([],[],pen=None,symbol='o')

### Helper Functions

In [2]:
def predictStep(x, A):
    xPred = np.matmul(A,x)
    return xPred

In [3]:
def innovationStep(xPred,z, H):
    innovation = np.subtract(z, np.matmul(H, xPred)) #innovation
    return innovation

In [4]:
def UpdateStep(xPred, nu, K):
    xNew = np.add(xPred, np.matmul(K, innovation))
    return xNew

In [48]:
def readMeasurements():
    #read in measurements from csv
    rawCentroidData = pd.read_csv('2PeopleCentroids.csv', header=None)
    #find headers and frames within headers
    #each header has the structure X, Y, CentroidNumber
    #below each header is the frame data
    centroidFramesCartesianMeasurement = list()
    headerFound = False
    for rowIndex in range(0, rawCentroidData.shape[0]):
        row = rawCentroidData.loc[rowIndex]
        if row[0] == 'X' and row[1] == 'Y' and row[2] == 'CentroidNumber':
            if headerFound: 
                #actual data was found last frame and this frame actual data is found again
                #past frame ended so add to list
                centroidFramesCartesianMeasurement.append(frame)
                frame = pd.DataFrame([])
            else:
                #header found
                headerFound = True
                frame = pd.DataFrame([])
            #data should be following
        elif headerFound:
            if np.isnan(np.float(row[0])) and np.isnan(np.float(row[1])):
                #empty row 
                centroidFramesCartesianMeasurement.append(frame)
                headerFound = False
                #only time its going to be NaN if the frame is completely empty
            else:
                #actual data
                X = np.float(row[0])
                Y = np.float(row[1])
                CentroidNumber = int(row[2])
                if len(frame) == 0:
                    frame = pd.DataFrame({'X':X, 'Y':Y, 'CentroidNumber':CentroidNumber}, index=range(1))
                else:
                    data = pd.DataFrame({'X':X, 'Y':Y, 'CentroidNumber':CentroidNumber}, index=range(CentroidNumber,CentroidNumber+1))
                    frame = pd.concat([frame,data])
    
    return centroidFramesCartesianMeasurement

In [43]:
#convert x,y to r,theta
def convertCartesianToPolar(cartesianMeasurementCentroidDf):
    measurementR = np.sqrt(cartesianMeasurementCentroidDf['X']**2 + cartesianMeasurementCentroidDf['Y']**2)
    measurementTheta = np.arctan2(cartesianMeasurementCentroidDf['Y'],cartesianMeasurementCentroidDf['X'])
    polarCentroidDf = pd.DataFrame({'CentroidNumber':np.arange(0,max(cartesianMeasurementCentroidDf['CentroidNumber'])+1), 
                                   'MeasuredRange': measurementR,
                                   'MeasuredTheta': measurementTheta})
    polarCentroidDf.index = range(0, polarCentroidDf.shape[0])
    return polarCentroidDf

In [7]:
def createDistanceMatrix(rThetaMeasurement,previousFrame):
    #create distance matrix
    distanceMatrix = np.eye(previousFrame.shape[0], rThetaMeasurement.shape[0])
    for centroid in previousFrame['CentroidNumber']:
        prevCentroidInfo = previousFrame.loc[centroid]
        #find distances between a selected prev centroid and the new centroids
        distances = np.sqrt(np.repeat(np.square(prevCentroidInfo.FilteredRange), repeats=rThetaMeasurement.shape[0]) + \
        np.square(rThetaMeasurement['MeasuredRange']) - \
        2*np.repeat(prevCentroidInfo.FilteredRange, repeats=rThetaMeasurement.shape[0]) * \
        rThetaMeasurement['MeasuredRange'] * np.cos(rThetaMeasurement['MeasuredTheta']-prevCentroidInfo.FilteredTheta))
    #     assign distances
        distanceMatrix[centroid,:] = distances
    return distanceMatrix

In [8]:
def dataAssociation(rThetaMeasurement, previousFrame):
    #create distance matrix
    distanceMatrix = createDistanceMatrix(rThetaMeasurement,previousFrame)
    association = np.full((max(distanceMatrix.shape[0], distanceMatrix.shape[1]), 2), np.nan)

    #associate
    numberOfLoops = min(distanceMatrix.shape[0], distanceMatrix.shape[1])
    for loopIterator in range(0, numberOfLoops):
        previousCentroid, measuredCentroid = np.where(np.min(distanceMatrix) == distanceMatrix)
    #     rThetaMeasurement.at[measuredCentroid[0],'CentroidNumber'] = previousCentroid[0] #associate with old centroid 
        association[loopIterator,0]= previousCentroid[0] #fill association matrix
        association[loopIterator,1]= measuredCentroid[0] #fill association matrix
        distanceMatrix[previousCentroid[0], :] = np.Inf
        distanceMatrix[:,measuredCentroid[0]] = np.Inf

    if np.isnan(association).any(): #if any NaN's still in the association matrix - mismatch alert
        if distanceMatrix.shape[0] > distanceMatrix.shape[1]:
            #more predictions than observations
            unassociatedPredictions = [pred for pred in list(previousFrame['CentroidNumber']) if pred not in list(association[:,0])]
            association[np.isnan(association[:,0]),0] = unassociatedPredictions
        else:
            #more observations than predictions
            #find which observation has not been associated
            unassociatedObservations = [observation for observation in list(rThetaMeasurement['CentroidNumber']) if observation not in list(association[:,1])]
            association[np.isnan(association[:,1]),1] = unassociatedObservations
            
    associationDf = pd.DataFrame(association, columns=['Predicted', 'Measured'])
    
    return associationDf

### Main Code

In [9]:
#initialize variables
deltaT = 50*10**-3 #50ms

#system matrix
A = np.array([
    [1, deltaT, 0,0],
    [0,1,0,0],
    [0,0,1,deltaT],
    [0,0,0,1]
])
#state transition matrix
F = expm(A*deltaT)
#output matrix
H = np.array([[1,0,0,0],
              [0,0,1,0]])
#covariance matrices
Q = np.eye(4)
R = np.ones(2).reshape(-1,1)
Pc = np.eye(4)

Pd = np.add(np.matmul(np.matmul(A,Pc), np.transpose(A)), Q) #prediction covariance
S = np.add(R, np.matmul(np.matmul(H, Pd), np.transpose(H))) #innovation covariance
K = np.matmul(Pd, np.matmul(np.transpose(H), np.linalg.inv(S))) #kalman gain
Pupdate = np.subtract(Pd, np.matmul(np.matmul(K, S), np.transpose(K))) #update covariance

filteredFramesPolar = list()

In [50]:
#read in measurements
centroidFramesCartesianMeasurement= readMeasurements()
#filtered frames (polaar measurements)
filteredFramesPolar = list()

for centroidFrame in centroidFramesCartesianMeasurement:
    filteredFramePolar = np.array([])
    print(centroidFrame)
    #first frame that is valid
    if len(filteredFramesPolar) == 0 and len(centroidFrame) > 0: #kalman filter initialization
        rThetaMeasurement = convertCartesianToPolar(centroidFrame)
        for centroid in rThetaMeasurement['CentroidNumber']:
            x = np.expand_dims(np.array([0,0,0,0]), axis=1) #initialise 
            
            #predict
            xPred = predictStep(x, A)
            #measurement
            centroidInformation = rThetaMeasurement.loc[centroid]
            measurement = np.expand_dims(np.array([centroidInformation.MeasuredRange,centroidInformation.MeasuredTheta]),
                                         axis=1)
            innovation = innovationStep(xPred, measurement, H)
            #update
            xUpdate = UpdateStep(xPred, innovation, K)
            #add in centroid number
            print(xUpdate)
            xUpdate = np.vstack((xUpdate, centroid))
            #add value to array
            if len(filteredFramePolar) == 0:
                filteredFramePolar = xUpdate
            else:
                filteredFramePolar = np.vstack(filteredFramePolar,xUpdate)
        
    #valid frames
    elif len(centroidFrame) >= 0 and len(filteredFramesPolar) != 0:
        if len(centroidFrame) != 0:
            rThetaMeasurement = convertCartesianToPolar(centroidFrame)
        else:
            rThetaMeasurement = pd.DataFrame([], columns=['CentroidNumber', 'MeasuredRange', 'MeasuredTheta'])
            
        #perform data association
        associationDf = dataAssociation(rThetaMeasurement, filteredFramesPolar[-1])
        previousFrame = filteredFramesPolar[-1]
        
        for centroidIndex in associationDf.index:

            #predict
            predictedCentroid = associationDf.loc[centroidIndex]['Predicted']
            if np.isnan(predictedCentroid):
                x = np.expand_dims(np.array([0,0,0,0]), axis=1) #initialise 
            else:
                xPred = previousFrame.loc[previousFrame['CentroidNumber'] == predictedCentroid]
                x = np.expand_dims(xPred.values[0][:4], axis=1)

            xPred = predictStep(x, A)
            #measurement
            associatedMeasuredCentroid = associationDf.loc[centroidIndex]['Measured']
            if not(np.isnan(associatedMeasuredCentroid)):
                #if associated centroid exists
                centroidInformation = rThetaMeasurement.loc[associatedMeasuredCentroid]
                measurement = np.expand_dims(np.array([centroidInformation.MeasuredRange,centroidInformation.MeasuredTheta]),axis=1)
                innovation = innovationStep(xPred, measurement, H)
                #update
                xUpdate = UpdateStep(xPred, innovation, K)
            else:
                xUpdate = xPred
            #add in centroid number
            print(xUpdate)
            xUpdate = np.vstack((xUpdate, centroid))
            #add value to array
            if len(filteredFramePolar) == 0:
                filteredFramePolar = xUpdate
            else:
                filteredFramePolar = np.hstack((filteredFramePolar,xUpdate))  
    
    else:
        continue
        
    filteredPolarDf = pd.DataFrame(np.transpose(filteredFramePolar))
    filteredPolarDf.columns = ['FilteredRange', 'FilteredDoppler', 'FilteredTheta', 'FilteredAngularVelocity', 'CentroidNumber']
    filteredPolarDf['CentroidNumber'] = pd.Series(np.arange(filteredPolarDf.shape[0]))
    filteredFramesPolar.append(filteredPolarDf) 

Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
          X         Y  CentroidNumber
0 -1.590903  3.145848               0
[[2.13504903]
 [0.05330959]
 [0.64881263]
 [0.01620007]]
Empty DataFrame
Columns: []
Ind

1  1.363386  4.011583               1
[[2.907996  ]
 [0.06659507]
 [2.11816452]
 [0.04945053]]
[[4.1642802 ]
 [0.10158069]
 [1.17052622]
 [0.02880953]]
          X         Y  CentroidNumber
0 -1.481316  2.482294               0
[[2.89879559]
 [0.06628221]
 [2.1169321 ]
 [0.04935803]]
[[4.16935924]
 [0.10158069]
 [1.1719667 ]
 [0.02880953]]
          X         Y  CentroidNumber
0 -1.405767  2.422528               0
1  1.339017  4.157072               1
[[2.83185467]
 [0.06452802]
 [2.12759017]
 [0.04956253]]
[[4.29776133]
 [0.10465992]
 [1.18954223]
 [0.0292124 ]]
          X         Y  CentroidNumber
0 -1.433417  2.379177               0
1  1.583496  4.283775               1
[[2.79622983]
 [0.06355795]
 [2.13165144]
 [0.04960205]]
[[4.49467088]
 [0.10944585]
 [1.14431879]
 [0.02804676]]
          X         Y  CentroidNumber
0 -1.410173  2.348468               0
1  1.343785  4.274193               1
[[2.75997444]
 [0.06257335]
 [2.13220953]
 [0.04955406]]
[[4.45527723]
 [0.10832561]
 [1

          X        Y  CentroidNumber
0 -0.353756  1.31028               0
[[1.3689943 ]
 [0.02532658]
 [1.84629357]
 [0.03962992]]
[[5.26714148]
 [0.12165273]
 [1.34842624]
 [0.03130131]]
          X         Y  CentroidNumber
0 -0.309125  1.295361               0
[[1.35215873]
 [0.02487459]
 [1.82547879]
 [0.03906072]]
[[5.27322412]
 [0.12165273]
 [1.3499913 ]
 [0.03130131]]
Empty DataFrame
Columns: []
Index: []
[[1.35340246]
 [0.02487459]
 [1.82743183]
 [0.03906072]]
[[5.27930676]
 [0.12165273]
 [1.35155637]
 [0.03130131]]
          X         Y  CentroidNumber
0 -0.259733  1.287242               0
[[1.33840565]
 [0.02446909]
 [1.7951196 ]
 [0.03820516]]
[[5.28538939]
 [0.12165273]
 [1.35312143]
 [0.03130131]]
          X         Y  CentroidNumber
0 -0.265557  1.334499               0
[[1.36285591]
 [0.02504903]
 [1.76941536]
 [0.03751566]]
[[5.29147203]
 [0.12165273]
 [1.3546865 ]
 [0.03130131]]
          X         Y  CentroidNumber
0 -0.224973  1.298077               0
[[1.33630849]


[[2.17121381]
 [0.04316906]
 [0.89664707]
 [0.01412185]]
          X         Y  CentroidNumber
0 -0.998259  4.850642               0
[[4.96500437]
 [0.10529145]
 [1.78646897]
 [0.03983727]]
[[2.17337226]
 [0.04316906]
 [0.89735316]
 [0.01412185]]
          X         Y  CentroidNumber
0  1.332222  2.153387               0
[[2.41340825]
 [0.04910858]
 [0.89800386]
 [0.01412046]]
[[4.97026895]
 [0.10529145]
 [1.78846083]
 [0.03983727]]
          X         Y  CentroidNumber
0  1.576138  2.229633               0
1 -0.967808  4.739483               1
[[4.87638107]
 [0.10281574]
 [1.81132125]
 [0.04035834]]
[[2.63769227]
 [0.05464737]
 [0.86267193]
 [0.01322064]]
          X         Y  CentroidNumber
0  1.471211  2.258737               0
1 -1.126094  4.727682               1
[[4.86751125]
 [0.10246591]
 [1.8121974 ]
 [0.04032983]]
[[2.64931216]
 [0.05486928]
 [0.94717252]
 [0.01531401]]
          X         Y  CentroidNumber
0  1.494583  2.275954               0
1 -1.110362  4.688828          

[[3.53277667]
 [0.06450251]
 [2.02578207]
 [0.04335219]]
[[4.0280906 ]
 [0.0857383 ]
 [1.06574699]
 [0.01737196]]
          X         Y  CentroidNumber
0 -1.385042  3.137241               0
[[3.46636071]
 [0.06276366]
 [2.02353008]
 [0.04324184]]
[[4.03237752]
 [0.0857383 ]
 [1.06661559]
 [0.01737196]]
          X         Y  CentroidNumber
0 -1.361004  3.097271               0
[[3.41490424]
 [0.0614005 ]
 [2.01661409]
 [0.04301517]]
[[4.03666443]
 [0.0857383 ]
 [1.06748419]
 [0.01737196]]
          X         Y  CentroidNumber
0 -1.437469  3.043083               0
1  1.820232  3.865556               1
[[3.38028501]
 [0.06045944]
 [2.02687165]
 [0.04321759]]
[[4.19920142]
 [0.08968961]
 [1.05723608]
 [0.01709439]]
          X         Y  CentroidNumber
0 -1.472114  2.983677               0
1  1.825394  3.830937               1
[[3.34109946]
 [0.05940555]
 [2.04316283]
 [0.04357041]]
[[4.21662731]
 [0.09001275]
 [1.09916225]
 [0.0181199 ]]
          X         Y  CentroidNumber
0 -1.497694 

...

 [0.01218263]]
          X         Y  CentroidNumber
0 -0.824873  1.374502               0
[[ 1.64761776]
 [-0.00717623]
 [ 2.15590573]
 [ 0.033413  ]]
[[5.01707982]
 [0.08048263]
 [1.29892701]
 [0.01218263]]
          X         Y  CentroidNumber
0 -0.770351  1.332374               0
[[ 1.58171242]
 [-0.00881284]
 [ 2.1376833 ]
 [ 0.03291629]]
[[5.02110395]
 [0.08048263]
 [1.29953614]
 [0.01218263]]
Empty DataFrame
Columns: []
Index: []
[[ 1.58127178]
 [-0.00881284]
 [ 2.13932911]
 [ 0.03291629]]
[[5.02512808]
 [0.08048263]
 [1.30014527]
 [0.01218263]]
          X         Y  CentroidNumber
0 -0.707381  1.268445               0
[[ 1.49981105]
 [-0.01083582]
 [ 2.12696774]
 [ 0.03256655]]
[[5.02915222]
 [0.08048263]
 [1.30075441]
 [0.01218263]]
          X        Y  CentroidNumber
0 -0.695034  1.32963               0
[[ 1.51908638]
 [-0.01034101]
 [ 2.07121729]
 [ 0.03113387]]
[[5.03317635]
 [0.08048263]
 [1.30136354]
 [0.01218263]]
Empty DataFrame
Columns: []
Index: []
[[ 1.51856933]
 [

 [0.01218263]]
Empty DataFrame
Columns: []
Index: []
[[ 1.52105061]
 [-0.00962646]
 [ 1.34204239]
 [ 0.01174953]]
[[5.21426226]
 [0.08048263]
 [1.32877446]
 [0.01218263]]
Empty DataFrame
Columns: []
Index: []
[[ 1.52056929]
 [-0.00962646]
 [ 1.34262987]
 [ 0.01174953]]
[[5.21828639]
 [0.08048263]
 [1.32938359]
 [0.01218263]]
Empty DataFrame
Columns: []
Index: []
[[ 1.52008797]
 [-0.00962646]
 [ 1.34321734]
 [ 0.01174953]]
[[5.22231053]
 [0.08048263]
 [1.32999273]
 [0.01218263]]
         X         Y  CentroidNumber
0  0.73411  1.589974               0
[[ 1.74474561]
 [-0.00400501]
 [ 1.13172454]
 [ 0.00645414]]
[[5.22633466]
 [0.08048263]
 [1.33060186]
 [0.01218263]]
Empty DataFrame
Columns: []
Index: []
[[ 1.74454536]
 [-0.00400501]
 [ 1.13204725]
 [ 0.00645414]]
[[5.23035879]
 [0.08048263]
 [1.33121099]
 [0.01218263]]
Empty DataFrame
Columns: []
Index: []
[[ 1.74434511]
 [-0.00400501]
 [ 1.13236995]
 [ 0.00645414]]
[[5.23438292]
 [0.08048263]
 [1.33182012]
 [0.01218263]]
Empty DataFra

[[4.14511489]
 [0.04951522]
 [1.94001637]
 [0.0260681 ]]
[[2.39055679]
 [0.01184221]
 [0.99672419]
 [0.00282455]]
          X         Y  CentroidNumber
0 -1.557743  3.849587               0
[[4.14801607]
 [0.04952584]
 [1.95050659]
 [0.02629748]]
[[2.3911489 ]
 [0.01184221]
 [0.99686542]
 [0.00282455]]
          X         Y  CentroidNumber
0 -1.470213  3.840706               0
[[4.12583897]
 [0.04891027]
 [1.94973764]
 [0.02624545]]
[[2.39174101]
 [0.01184221]
 [0.99700664]
 [0.00282455]]
          X         Y  CentroidNumber
0 -1.495077  3.786708               0
[[4.08649251]
 [0.04786678]
 [1.96215539]
 [0.02652274]]
[[2.39233312]
 [0.01184221]
 [0.99714787]
 [0.00282455]]
          X         Y  CentroidNumber
0 -1.546165  3.776994               0
[[4.08416362]
 [0.04774887]
 [1.96229803]
 [0.02649319]]
[[2.39292524]
 [0.01184221]
 [0.9972891 ]
 [0.00282455]]
          X         Y  CentroidNumber
0 -1.612512  3.699369               0
[[4.04372479]
 [0.04667955]
 [1.99004497]
 [0.0271

[[2.49524414]
 [0.00639016]
 [2.1937057 ]
 [0.03051257]]
[[4.76784164]
 [0.06902806]
 [1.2835743 ]
 [0.0097186 ]]
          X         Y  CentroidNumber
0 -1.414654  2.044910               0
1  1.286295  4.641006               1
[[2.49360402]
 [0.00634123]
 [2.18305305]
 [0.0302085 ]]
[[4.8007131 ]
 [0.06976264]
 [1.2851756 ]
 [0.00974645]]
          X         Y  CentroidNumber
0 -1.362407  1.990323               0
1  1.360416  4.667063               1
[[4.84665748]
 [0.07082273]
 [1.27252258]
 [0.00941835]]
[[2.43581068]
 [0.00489029]
 [2.1949075 ]
 [0.03046677]]
          X         Y  CentroidNumber
0 -1.269316  1.927053               0
1  1.343337  4.693748               1
[[4.8694397 ]
 [0.07130315]
 [1.27929506]
 [0.00957569]]
[[2.35042971]
 [0.00275232]
 [2.19615036]
 [0.03045977]]
          X         Y  CentroidNumber
0 -1.349219  1.947532               0
1  1.524618  4.755724               1
[[2.36981986]
 [0.00323304]
 [2.1772485 ]
 [0.02994979]]
[[4.96867023]
 [0.0736918 ]
 [1

[[5.24542053]
 [0.07625582]
 [1.35983009]
 [0.01099127]]
[[ 1.71990175]
 [-0.01252882]
 [ 2.2787925 ]
 [ 0.0307238 ]]
Empty DataFrame
Columns: []
Index: []
[[5.24923332]
 [0.07625582]
 [1.36037966]
 [0.01099127]]
[[ 1.71927531]
 [-0.01252882]
 [ 2.28032869]
 [ 0.0307238 ]]
Empty DataFrame
Columns: []
Index: []
[[5.25304611]
 [0.07625582]
 [1.36092922]
 [0.01099127]]
[[ 1.71864887]
 [-0.01252882]
 [ 2.28186488]
 [ 0.0307238 ]]
Empty DataFrame
Columns: []
Index: []
[[5.2568589 ]
 [0.07625582]
 [1.36147878]
 [0.01099127]]
[[ 1.71802243]
 [-0.01252882]
 [ 2.28340107]
 [ 0.0307238 ]]
Empty DataFrame
Columns: []
Index: []
[[5.26067169]
 [0.07625582]
 [1.36202835]
 [0.01099127]]
[[ 1.71739598]
 [-0.01252882]
 [ 2.28493726]
 [ 0.0307238 ]]
Empty DataFrame
Columns: []
Index: []
[[5.26448448]
 [0.07625582]
 [1.36257791]
 [0.01099127]]
[[ 1.71676954]
 [-0.01252882]
 [ 2.28647346]
 [ 0.0307238 ]]
Empty DataFrame
Columns: []
Index: []
[[5.26829727]
 [0.07625582]
 [1.36312747]
 [0.01099127]]
[[ 1.71

In [ ]:
#plot
xPositions = np.array([])
yPositions = np.array([])
for centroidFrame in filteredFramesPolar:
    try:
        x = np.multiply(centroidFrame.FilteredRange[1], np.cos(centroidFrame.FilteredTheta[1]))
        y = np.multiply(centroidFrame.FilteredRange[1], np.sin(centroidFrame.FilteredTheta[1]))
    except:
        continue
    if len(xPositions) == 0:
        xPositions = np.array([x])
    else:
        xPositions = np.append(xPositions, x)       
    if len(yPositions) == 0:
        yPositions = np.array([y])
    else:
        yPositions = np.append(yPositions,y)

    s1.setData(xPositions, yPositions)
    QtGui.QApplication.processEvents() 
    time.sleep(0.1)

In [69]:
len(filteredFramesPolar)

879

3.411749496326833

## Unit Tests

In [162]:
#test centroid dataset
#centroidDf_A contains 2 centroids
centroidDfCartesian_A = pd.DataFrame({'CentroidNumber':np.arange(0,2), 
                             'X':np.array([-1,0]), 
                             'Y':np.array([3,1])})
#centroidDf_B contains 2 centroids and continues on from centroidDf_A
centroidDfCartesian_B = pd.DataFrame({'CentroidNumber':np.arange(0,2), 
                             'X':np.array([0.5,-1]), 
                             'Y':np.array([0.7,2.5])})
#centroidDf_C contains 3 centroids continuing from centroidDf_B
centroidDfCartesian_C = pd.DataFrame({'CentroidNumber':np.arange(0,3), 
                             'X':np.array([-1,0.7, 0.3]), 
                             'Y':np.array([1.2,0.5, 3.2])})

centroidDfCartesian_D = pd.DataFrame([])

#contains the centroid frames, where each frame represents a mmWave capture frame
centroidFramesCartesianMeasurement = [centroidDfCartesian_A, centroidDfCartesian_B,centroidDfCartesian_D]

In [310]:
previousFrame = filteredFramesPolar[-1]
rThetaMeasurement = convertCartesianToPolar(centroidDfCartesian_C)
filteredFramePolar = np.array([])

distanceMatrix = createDistanceMatrix(rThetaMeasurement,previousFrame)
association = np.full((max(distanceMatrix.shape[0], distanceMatrix.shape[1]), 2), np.nan)

#associate
numberOfLoops = min(distanceMatrix.shape[0], distanceMatrix.shape[1])
for loopIterator in range(0, numberOfLoops):
    previousCentroid, measuredCentroid = np.where(np.min(distanceMatrix) == distanceMatrix)
#     rThetaMeasurement.at[measuredCentroid[0],'CentroidNumber'] = previousCentroid[0] #associate with old centroid 
    association[loopIterator,0]= previousCentroid[0] #fill association matrix
    association[loopIterator,1]= measuredCentroid[0] #fill association matrix
    distanceMatrix[previousCentroid[0], :] = np.Inf
    distanceMatrix[:,measuredCentroid[0]] = np.Inf

if np.isnan(association).any(): #if any NaN's still in the association matrix - mismatch alert
    if distanceMatrix.shape[0] > distanceMatrix.shape[1]:
        #more predictions than observations
        unassociatedPredictions = [pred for pred in list(previousFrame['CentroidNumber']) if pred not in list(association[:,0])]
        association[np.isnan(association[:,0]),0] = unassociatedPredictions
    else:
        #more observations than predictions
        #find which observation has not been associated
        unassociatedObservations = [observation for observation in list(rThetaMeasurement['CentroidNumber']) if observation not in list(association[:,1])]
        association[np.isnan(association[:,1]),1] = unassociatedObservations
        
associationDf = pd.DataFrame(association, columns=['Predicted', 'Measured'])

for centroidIndex in associationDf.index:
    
    #predict
    predictedCentroid = associationDf.loc[centroidIndex]['Predicted']
    if np.isnan(predictedCentroid):
        x = np.expand_dims(np.array([0,0,0,0]), axis=1) #initialise 
    else:
        xPred = previousFrame.loc[previousFrame['CentroidNumber'] == predictedCentroid]
        x = np.expand_dims(xPred.values[0][:4], axis=1)
    
    xPred = predictStep(x, A)
    #measurement
    associatedMeasuredCentroid = associationDf.loc[centroidIndex]['Measured']
    if not(np.isnan(associatedMeasuredCentroid)):
        #if associated centroid exists
        centroidInformation = rThetaMeasurement.loc[associatedMeasuredCentroid]
        measurement = np.expand_dims(np.array([centroidInformation.MeasuredRange,centroidInformation.MeasuredTheta]),axis=1)
        innovation = innovationStep(xPred, measurement, H)
        #update
        xUpdate = UpdateStep(xPred, innovation, K)
    else:
        xUpdate = xPred
    
    #add value to array
    if len(filteredFramePolar) == 0:
        filteredFramePolar = xUpdate
    else:
        filteredFramePolar = np.hstack((filteredFramePolar,xUpdate))
        
filteredPolarDf = pd.DataFrame(np.transpose(filteredFramePolar))
filteredPolarDf.columns = ['FilteredRange', 'FilteredDoppler', 'FilteredTheta', 'FilteredAngularVelocity']
filteredPolarDf['CentroidNumber'] = pd.Series(np.arange(filteredPolarDf.shape[0]))
filteredFramesPolar.append(filteredPolarDf)    

In [311]:
filteredPolarDf

,FilteredRange,FilteredDoppler,FilteredTheta,FilteredAngularVelocity,CentroidNumber
0,1.277990,0.031801,1.038007,0.025929,0
1,0.605751,0.015125,1.309236,0.032690,1
2,2.041926,0.050984,0.305214,0.007621,2


In [301]:
centroidIndex

1

In [304]:
associationDf

,Predicted,Measured
0,0.0,1.0
1,NaN,0.0
2,NaN,2.0
